# 2.2. Recursion and Tree Data-Types

In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";

// CSC 600 Libraries
import { drawList, drawTree, drawCallStack, requireCytoscape, requireCarbon} from "./lib/draw";
import * as introspect from "./lib/introspect";
import * as list from "./lib/list";

requireCarbon();
requireCytoscape();

## Where Were We?

Concept Roadmap:

1. **Bottom-up, i.e., building blocks of languages.** (TODAY and next 3 weeks)
    - **Data-Types + Recursion** (this week)
    - First-Class Functions or References + State
2. Top-down, i.e., using building blocks.
3. *Meta-theory.*

## Goal

1. Continue study of **(algebraic) data-types** (**ADT**) using **trees**.
2. Examine how **recursive functions** operate on trees.

Recall language template

1. Input/Output:
2. **Data: tree data-type**
3. **Code: recursive function**

## Outline

- Why ADTs?
- Trees

## Why ADTs?

Example:  [https://www.paulrand.design/work/Book-Covers.html](https://www.paulrand.design/work/Book-Covers.html)

Here's a question for you first.

```








Imagine you're a graphic designer.
You want to layout a book cover with:
1. Text
2. Drawings
in an aesthetically pleasing way.


How would you tackle the problem of laying out such a book cover?









```

In [2]:
tslab.display.html(`
<div class="bx--tile">
    BOX 1: Imagine I'm a page
</div>
`)

BOX 1: Imagine I'm a page

In [3]:
tslab.display.html(`
<div class="bx--grid"> BOX 1: Imagine I'm a page
    <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX 2: Left side of page
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX 3: Right side of page
            </div>
        </div>
    </div>
`)

BOX 1: Imagine I'm a page
 
 
 
 BOX 2: Left side of page
 
 
 
 
 BOX 3: Right side of page

In [4]:
tslab.display.html(`
<div class="bx--grid"> BOX 1: Imagine I'm a page
    <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX2: Left side of page
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX3: Right side of page
                
                <div class="bx--tile">
                    BOX4: A sub-page within the right page
                </div>
            </div>
        </div>
    </div>
</div>
`)

BOX 1: Imagine I'm a page
 
 
 
 BOX2: Left side of page
 
 
 
 
 BOX3: Right side of page
 
 
 BOX4: A sub-page within the right page

### Notice a Pattern?

- To solve the graphic design problem, we're "nesting" boxes within boxes.
- This is starting to look like recursion.
    - Maybe I don't know how to layout the full page.
    - I'll solve a simpler version of the same problem by splitting the page into different smaller pages.
    - Now let me solve the layout problem for the smaller pages.
    - And put together the results for the original page.
- But we need something more complex than the list ADT.

## Trees

- Enter the next simplest ADT, the *tree* data-types.

In [5]:
import * as tree from "./lib/tree";

drawTree(tree.t4)

In [6]:
// Literal types

const tag: 0 = 0;      // In typescript, you can use booleans, strings, and numbers as types
// const tag: 0 = 1;   // Fails because 1 is not 0

In [7]:
// Object types

const x: { key1: 0, key2: number} = { key1: 0, key2: 1};  //
// const fail2: { key1: 0, key2: number} = { key: 0, key2: 1};  // Fails because key is not key1
// const fail1: { key1: 0, key2: number} = { key1: 2, key2: 1}; // Fails because the value 2 is not 0

In [8]:
// Tree data-type
enum _Tree { LEAF, NODE };    // LEAF refers to 0, NODE refers to 1
type Tree<T> =
    {tag: _Tree.LEAF}         // Leaf node, similar to List Nil
  | {tag: _Tree.NODE, contents: T, left: Tree<T>, right: Tree<T>};  // Tree node, similar to List Cons

In [9]:
function Leaf<T>(): Tree<T> {
    // Constructor function for a node with no contents.
    return {tag: _Tree.LEAF};
}

function Node<T>(x: T, left: Tree<T>, right: Tree<T>): Tree<T> {
    // Constructor function for a node with contents, a left sub-tree, and a right-subtree.
    return {tag: _Tree.NODE, contents: x, left: left, right: right};
}

function LeafNode<T>(x: T): Tree<T> {
    return Node(x, Leaf(), Leaf());
}

In [10]:
const t0 = LeafNode("Box2: Left")
const t1 = LeafNode("Box 4: Right Bot");
const t2 = Node("Box 3: Right", t1, Leaf());
const t3 = Node("Box 1", t0, t2);

In [11]:
drawTree(t1)

In [12]:
drawTree(t2)

In [13]:
drawTree(t3)

In [14]:
t1

{
  tag: 1,
  contents: 'Box 4: Right Bot',
  left: { tag: 0 },
  right: { tag: 0 }
}


In [15]:
t2 // Notice nesting of t1 in t2's left child

{
  tag: 1,
  contents: 'Box 3: Right',
  left: {
    tag: 1,
    contents: 'Box 4: Right Bot',
    left: { tag: 0 },
    right: { tag: 0 }
  },
  right: { tag: 0 }
}


In [16]:
t3 // Notice nesting of t2 in t3's right child

{
  tag: 1,
  contents: 'Box 1',
  left: {
    tag: 1,
    contents: 'Box2: Left',
    left: { tag: 0 },
    right: { tag: 0 }
  },
  right: {
    tag: 1,
    contents: 'Box 3: Right',
    left: {
      tag: 1,
      contents: 'Box 4: Right Bot',
      left: [Object],
      right: [Object]
    },
    right: { tag: 0 }
  }
}


#### Recursive Functions on Trees

In [17]:
// Compare with length on tree
function height<T>(t: Tree<T>): number {
    switch (t.tag) {
        case _Tree.LEAF: {   // Base case: leaf node
            return 0;
        }
        case _Tree.NODE: {   // Recursive (or inductive) case: node with left and right child
            // Compare with length on tree
            return 1 + Math.max(height(t.left), height(t.right));
        }
    }
}

height(t3)

3


In [18]:
// Compare with list
const res = introspect.traceCallStack(height, exports);
res.func(t1)
drawCallStack(res.stack)

In [19]:
// Compare with list
const res = introspect.traceCallStack(height, exports);
res.func(t2) // recall nesting again
drawCallStack(res.stack)

In [20]:
function treeToString<T>(t: Tree<T>): string {
    switch (t.tag) {
        case _Tree.LEAF: {
            return "()";
        }
        case _Tree.NODE: {
            return `(${t.contents.toString()} ${treeToString(t.left)} ${treeToString(t.right)})`;
        }
            
    }
}

In [21]:
treeToString(t3)

(Box 1 (Box2: Left () ()) (Box 3: Right (Box 4: Right Bot () ()) ()))


In [22]:
const res = introspect.traceCallStack(treeToString, exports);
res.func(t2)
drawCallStack(res.stack)

#### What do the iterative solutions look like?

In [23]:
function iterTreeToString<T>(t: Tree<T>): string {
    let callStack: [string, Tree<T> | string][] = [];
    let ans: string[] = [];
    callStack.push(["CALL", t]);
    
    while (callStack.length > 0) {
        const [mode, trOrStr] = callStack.pop();
        if (mode == "CALL") {
            const tr = trOrStr as Tree<T>;
            switch ((tr).tag) {
                case _Tree.LEAF: {
                    ans.push("()");
                    break;
                }
                case _Tree.NODE: {
                    callStack.push(["PROCESS", tr.contents.toString()]);
                    callStack.push(["CALL", tr.right]);
                    callStack.push(["CALL", tr.left]);
                    break;
                }
            }
        } else if (mode == "PROCESS") {
            const str = trOrStr as string;
            const right = ans.pop();
            const left = ans.pop();
            ans.push(`(${str} ${left} ${right})`);
        } else {
            throw Error("Shouldn't happen ...");
        }
    }
    
    return ans.pop();
}

In [24]:
[iterTreeToString(t2), treeToString(t2)]

[
  '(Box 3: Right (Box 4: Right Bot () ()) ())',
  '(Box 3: Right (Box 4: Right Bot () ()) ())'
]


In [25]:
[iterTreeToString(t3), treeToString(t3)]

[
  '(Box 1 (Box2: Left () ()) (Box 3: Right (Box 4: Right Bot () ()) ()))',
  '(Box 1 (Box2: Left () ()) (Box 3: Right (Box 4: Right Bot () ()) ()))'
]


### Summary

- You can do the **same** computations with both iteration and recursion.
- Recursion uses more **stack frames** (the traceCallStack picture) than iteration.
- For some functions, iteration requires you to simulate the stack on the heap (e.g., iterTreeToString)

## JSON

[https://www.json.org/json-en.html](https://www.json.org/json-en.html)

In [26]:
// Our tree example in JSON

const j0 = {'tag': 'div', 'style': '', 'body': 'BOX 2: Left', 'children': []};
const j1 = {'tag': 'div', 'style': '', 'body': 'BOX 4: Right Bot', 'children': []};
const j2 = {'tag': 'div', 'style': '', 'body': 'BOX 3: Right', 'children': [j1]};
const j3 = {'tag': 'div', 'style': '', 'body': 'BOX 1: Right', 'children': [j0, j2]};

j3

{
  tag: 'div',
  style: '',
  body: 'BOX 1: Right',
  children: [
    { tag: 'div', style: '', body: 'BOX 2: Left', children: [] },
    { tag: 'div', style: '', body: 'BOX 3: Right', children: [Array] }
  ]
}


### JSON Data Interchange Format

In [27]:
type JSONValue = null | string | JSONObject | JSONValue[];
type JSONObject = { [key: string]: JSONValue };

function treeToJSON<T>(tr: Tree<T>): JSONObject {
    switch (tr.tag) {
        case (_Tree.LEAF): {
            return { 'tag': '_Tree.LEAF' }
        }
        case (_Tree.NODE): {
            return { 'tag': '_Tree.NODE',
                     'contents': tr.contents.toString(),
                     'left': treeToJSON(tr.left),
                     'right': treeToJSON(tr.right) }
        }
    }
}

In [28]:
import * as fs from "fs";

const jsonTree = treeToJSON(t3);
console.log("Original", JSON.stringify(jsonTree));

fs.writeFileSync('jsonTree.json', JSON.stringify(jsonTree));
try {
    const data = fs.readFileSync('jsonTree.json', 'utf8')
    console.log("Decoded", data);
    console.log("Decoded successfully?", data == JSON.stringify(jsonTree))
} catch (err) {
    tslab.display.html(`<p> ${err}</p>`);
}

Original {"tag":"_Tree.NODE","contents":"Box 1","left":{"tag":"_Tree.NODE","contents":"Box2: Left","left":{"tag":"_Tree.LEAF"},"right":{"tag":"_Tree.LEAF"}},"right":{"tag":"_Tree.NODE","contents":"Box 3: Right","left":{"tag":"_Tree.NODE","contents":"Box 4: Right Bot","left":{"tag":"_Tree.LEAF"},"right":{"tag":"_Tree.LEAF"}},"right":{"tag":"_Tree.LEAF"}}}
Decoded {"tag":"_Tree.NODE","contents":"Box 1","left":{"tag":"_Tree.NODE","contents":"Box2: Left","left":{"tag":"_Tree.LEAF"},"right":{"tag":"_Tree.LEAF"}},"right":{"tag":"_Tree.NODE","contents":"Box 3: Right","left":{"tag":"_Tree.NODE","contents":"Box 4: Right Bot","left":{"tag":"_Tree.LEAF"},"right":{"tag":"_Tree.LEAF"}},"right":{"tag":"_Tree.LEAF"}}}
Decoded successfully? true


In [29]:
function jsonToTree(jTree: JSONObject): Tree<string> {
    switch ((jTree as { [tag: string]: any }).tag) {
        case ('_Tree.LEAF'): {
            return Leaf();
        }
        case ('_Tree.NODE'): {
            const node = jTree as { tag: string, contents: string, left: JSONObject, right: JSONObject }
            return Node(node.contents, jsonToTree(node.left), jsonToTree(node.right));
        }
    }
}

drawTree(jsonToTree(treeToJSON(t3)))

### JSON's as serialization format for TypeScript object

In [37]:
t3 // Notice how similar this is to JSON

{
  tag: 1,
  contents: 'Box 1',
  left: {
    tag: 1,
    contents: 'Box2: Left',
    left: { tag: 0 },
    right: { tag: 0 }
  },
  right: {
    tag: 1,
    contents: 'Box 3: Right',
    left: {
      tag: 1,
      contents: 'Box 4: Right Bot',
      left: [Object],
      right: [Object]
    },
    right: { tag: 0 }
  }
}


In [31]:
console.log("Original", JSON.stringify(t3));

fs.writeFileSync('jsonTree.json', JSON.stringify(t3));
try {
    const data = fs.readFileSync('jsonTree.json', 'utf8')
    console.log("Decoded", data);
    console.log("Decoded successfully?", data == JSON.stringify(t3))
} catch (err) {
    tslab.display.html(`<p> ${err}</p>`);
}

Original {"tag":1,"contents":"Box 1","left":{"tag":1,"contents":"Box2: Left","left":{"tag":0},"right":{"tag":0}},"right":{"tag":1,"contents":"Box 3: Right","left":{"tag":1,"contents":"Box 4: Right Bot","left":{"tag":0},"right":{"tag":0}},"right":{"tag":0}}}
Decoded {"tag":1,"contents":"Box 1","left":{"tag":1,"contents":"Box2: Left","left":{"tag":0},"right":{"tag":0}},"right":{"tag":1,"contents":"Box 3: Right","left":{"tag":1,"contents":"Box 4: Right Bot","left":{"tag":0},"right":{"tag":0}},"right":{"tag":0}}}
Decoded successfully? true


### JSON as ADT

Recall the type of `JSONObject`

```
type JSONPrimitive = string | JSONObject | undefined;
type JSONObject = { [key: string]: JSONPrimitive } | JSONObject[];
```

Looks like we can change this to an ADT.

In [35]:
// JSON as ADT

type JSONValue = null | string | JSONObject | JSONValue[];
type JSONObject = { [key: string]: JSONValue };

And finally ADTs look like BNFS ...

\begin{align*}
\text{string} & ::= \dots \\
\text{value} & ::= \texttt{null} \,\,|\,\, \text{<string>} \,\,|\,\, \text{<object>} \,\,|\,\, [\text{<value>}] \\
\text{member} & ::= \text{string} \, \texttt{:} \, \text{value} \\
\text{object} & ::= \texttt{\{} [\text{member}] \texttt{\}} \\
\end{align*}

In [33]:
tslab.display.html(`
<div class="bx--grid"> Program 1: I'm a top-level program
    <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX2: Sub-program 2
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX3: Sub-program 3
                
                <div class="bx--tile">
                    BOX4: A sub-sub-program
                </div>
            </div>
        </div>
    </div>
</div>
`)

Program 1: I'm a top-level program
 
 
 
 BOX2: Sub-program 2
 
 
 
 
 BOX3: Sub-program 3
 
 
 BOX4: A sub-sub-program

## Story for Today?

- We motivated ADTs + recursion as a natural way of solving a problem such as page layout.
- This led use to realize that we needed a general way to construct data-structures which turned out to be ADTs.
- We looked at the next simplest example of an ADT called a Tree.
- Trees are surprisingly powerful. In particular, we saw that JSON is a "Tree".
- And in fact every TypeScript object underneath looks a lot like JSON.
- We'll continue to see the concept of recursion throughout the class.